In [719]:
import numpy as np
import pandas as pd

### Pdf reading

In [720]:
%pip install --upgrade pdfplumber pdfminer -q

import pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [721]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    pdf = pdfplumber.open(path)
    for page in range(len(pdf.pages)):
        # get the page object
        page_obj = pdf.pages[page]
        # extract text from the page
        text.append(
            page_obj.extract_text(
                x_tolerance=1, x_tolerance_ratio=None, y_tolerance=1, layout=False, x_density=7.25, y_density=13, line_dir_render=None,keep_blank_chars=True,use_text_flow=True, char_dir_render=None, expand_ligatures=True
            )
        )
    text = "\n".join(text)
    
    return text

In [722]:
path = "D:\Projects\LLM\Createdb\docs\\aco and pso.pdf"
text = _convert_pdf_to_text(path)

KeyboardInterrupt: 

In [ ]:
print(text)

Journal of Constructional Steel Research 65 (2009) 1558–1568
Contents lists available at ScienceDirect
Journal of Constructional Steel Research
journal homepage: www.elsevier.com/locate/jcsr
A particle swarm ant colony optimization for truss structures with
discrete variables
A. Kaveh
a,∗
,1
, S. Talatahari
b
a
Institute for Mechanics of Materials and Structures, Vienna University of Technology, Karlsplatz 13, A-1040 Wien, Austria
b
Department of Civil Engineering, Tabriz University, Tabriz, Iran
a r t i c l e i n f o
Article history:
Received 19 March 2009
Accepted 24 April 2009
Keywords:
Heuristic particle swarm ant colony
optimization
Harmony search
Optimum truss design
Discrete variables
a b s t r a c t
In this paper, a particle swarm optimizer with passive congregation (PSOPC), ant colony optimization
(ACO) and harmony search scheme (HS) are combined to reach to an efficient algorithm, called discrete
heuristic particle swarm ant colony optimization (DHPSACO). This method is then 

In [ ]:
data = [[text]]

In [ ]:
df = pd.DataFrame(data, columns=['raw_text'])

### adding title from first page

In [ ]:
import pdfplumber

def find_text_by_largest_font(page):
    largest_font_size = 0
    largest_font_chars = []

    for char in page.chars:
        size = char["size"]
        if size > largest_font_size:
            largest_font_size = size
            largest_font_chars = [char]
        elif size == largest_font_size:
            largest_font_chars.append(char)
    # for char in page.chars:
    #     size = char["size"]
    #     if size == largest_font_size or char == " ":
    #         largest_font_chars.append(char)

    return largest_font_chars


with pdfplumber.open(path) as pdf:
    first_page = pdf.pages[0]
    largest_font_chars = find_text_by_largest_font(first_page)

    largest_font_text = ''.join(char['text'] for char in largest_font_chars)

    # print("-----")
    # print(largest_font_text)
    # print("-----")
df['Title'] = largest_font_text

In [ ]:
df

,raw_text,Title
0,Journal of Constructional Steel Research 65 (2...,JournalofConstructionalSteelResearch


In [ ]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\sai18\AppData\Local\Temp\ipykernel_25100\2340421553.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### header

In [ ]:
def raw(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[1] 
    else:
        return ''

def header(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[0]   
    else:
        return ''

In [ ]:
import re
def abstract(text):
    pattern = re.compile(r'(\s*)a(\s*)b(\s*)s(\s*)t(\s*)r(\s*)a(\s*)c(\s*)t(\s*)', re.IGNORECASE)
    matches = pattern.findall(text)
    if len(matches) == 0:
        return ""
    delimiter = ''.join(match[0] for match in matches)

    new = text.split(delimiter)
    print(new[0])
    print(new[1])
    return new[1]

In [ ]:
df['header'] = df['raw_text'].apply(header)

In [ ]:
df['raw_text'] = df['raw_text'].apply(raw)

In [ ]:
df['abstract'] = df['header'].apply(abstract)

### Punctuation removal

In [ ]:
import string
import re 
# import nltk
# from nltk.corpus import stopwords



def pc(data_text):
    data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
    text = ""
    for i in data_text:
        text = text + i
    print(len(text))

    text = ""

    for i in data_text:
        text = text + i
    
    return text

df['pc_removal'] = df['raw_text'].apply(pc)


0


### url

In [ ]:
# #url
url_pattern = r'(?:https?://)?(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?'
def add_urls(text):
    httplinks = re.findall(url_pattern,text)
    return httplinks

url_pattern = re.compile(url_pattern)
def remove_urls(text):
    return url_pattern.sub('', text)

df['urls'] = df['pc_removal'].apply(add_urls)
df['without_url'] = df['pc_removal'].apply(remove_urls)

### 

### split refrences

In [ ]:
df['without_refs'] = df['without_url'][0].split('references')[0]
df['refs'] = df['without_url'][0].split('references')[1]

IndexError: list index out of range

### numbers

In [ ]:
import re

number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

def remove_number(text):
    return number_pattern.sub('', text)

# Assuming df['without_url'] is your DataFrame column containing text
df['without_numbers'] = df['without_refs'].apply(remove_number)


### mail

In [ ]:
# mail
mail_pattern = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
def remove_mail(text):
    return mail_pattern.sub('', text)
df['without_mail'] = df['without_numbers'].apply(remove_mail)
# df['without_mail'] = df['without_refs'].apply(remove_mail)

### sentence segmentation

In [ ]:
def sen_seg(data_text):
    sentences = []

    sentence = ''

    for i in range(len(data_text)):
        if data_text[i] == ".":
            sentences.append(sentence)
            sentence = ""
            continue
        else:
            sentence += data_text[i]

    sentences1 = []
    for i in sentences:
        if("\n" in i):
            # print("Yes")
            i = i.replace("\n", " ")
        # print(i)
        sentences1.append(i)
    
    return sentences1

df['sentences'] = df['without_mail'].apply(sen_seg)


In [ ]:
print(len(df['sentences'][0]))

313


In [ ]:
def checker(sentences):
    new = []
    for s in sentences:
        if(len(s) > 30):
            new.append(s)

    return new

df['sentences'] = df['sentences'].apply(checker)

## String Slicing

In [ ]:
from nltk.tokenize import word_tokenize

new = word_tokenize(text)

In [ ]:
# import numpy as np
# from wordfreq import get_frequency_dict

# word_prob = get_frequency_dict(lang='en', wordlist='large')
# max_word_len = max(map(len, word_prob))  # 34

# def str_slice(text):
#     def viterbi_segment(text, debug=False):
#         probs, lasts = [1.0], [0]
#         for i in range(1, len(text) + 1):
#             new_probs = []
#             for j in range(max(0, i - max_word_len), i):
#                 substring = text[j:i]
#                 length_reward = np.exp(len(substring))
#                 freq = word_prob.get(substring, 0) * length_reward
#                 compounded_prob = probs[j] * freq
#                 new_probs.append((compounded_prob, j))
                
#                 if debug:
#                     print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

#             prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
#             probs.append(prob_k)
#             lasts.append(k)

#             if debug:
#                 print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


#         # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
#         # in that case, return the original word instead
#         if len(set(lasts)) == len(text):
#             return text

#         words = []
#         k = len(text)
#         while 0 < k:
#             word = text[lasts[k]:k]
#             words.append(word)
#             k = lasts[k]
#         words.reverse()
#         return ' '.join(words)

#     def split_message(message):
#         new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
#         return new_message
    
#     new_sentences = []

#     for s in text:
#         new_msg = split_message(s)
#         new_sentences.append(new_msg)

#     return new_sentences

In [ ]:
# df['string_slicing'] = df['sentences'].apply(str_slice)

In [ ]:
df

,raw_text,Title,header,abstract,pc_removal,urls,without_url,without_refs,refs,without_numbers,without_mail,sentences
0,\nstructural optimization has become one of th...,journalofconstructionalsteelresearch,journal of constructional steel research 65 (2...,contents lists available at sciencedirect,\nstructural optimization has become one of th...,"[iust.ac, j.jcsr]",\nstructural optimization has become one of th...,\nstructural optimization has become one of th...,"\n[1] kaveh a, farahmand azar b, talatahari s....",\nstructural optimization has become one of th...,\nstructural optimization has become one of th...,[ structural optimization has become one of th...
